In [15]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd
import csv
import time

In [16]:
df = pd.read_csv('graph_data.csv')
display(df)

,n,m,Edges
0,10,10,"(1, 9)"
1,10,10,"(1, 3)"
2,10,10,"(2, 5)"
3,10,10,"(3, 5)"
4,10,10,"(4, 8)"
...,...,...,...
995,20,180,"(17, 20)"
996,20,180,"(17, 18)"
997,20,180,"(18, 19)"
998,20,180,"(18, 20)"


In [17]:
graph_data = {}
for index, row in df.iterrows():
    n = row['n']
    m = row['m']
    edge_tuple = tuple(map(int, row['Edges'].strip("()").split(",")))

    if n not in graph_data:
        graph_data[n] = {}

    if m not in graph_data[n]:
        graph_data[n][m] = []

    graph_data[n][m].append(edge_tuple)

In [18]:
def lp_rounding(n, edges, weights=None):

    if weights is None:
        weights = [1] * n

    c = np.array(weights)
    A = []
    b = []

    # Constraints
    for u, v in edges:
        constraint = np.zeros(n)
        constraint[u-1] = -1
        constraint[v-1] = -1
        A.append(constraint)
        b.append(-1)

    A = np.array(A)
    b = np.array(b)
    bounds = [(0, 1) for _ in range(n)]

    t0 = time.perf_counter()
    res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')
    t1 = time.perf_counter()
    lp_fractional_time = t1 - t0

    x_star = res.x
    t2 = time.perf_counter()
    cover = [i for i in range(n) if x_star[i] >= 0.5]
    t3 = time.perf_counter()
    lp_rounding_time = t3 - t2

    return cover, x_star, lp_fractional_time, lp_rounding_time


In [19]:
results = {}

for n in graph_data:
    results[n] = {}
    for m in graph_data[n]:
        edges = graph_data[n][m]

        cover, x, frac_time, round_time = lp_rounding(n, edges)
        # Compute LP optimal value(sum of fractional values)
        lp_optimal = np.sum(x)

        results[n][m] = {
            'fractional_values': x,
            'lp_rounding_cover': cover,
            'lp_optimal': lp_optimal,
            'approximation_cover_size': len(cover),
            'lp_fractional_time': frac_time,
            'lp_rounding_time': round_time
        }


In [21]:
approx_results = pd.read_csv('results_data.csv')
lp_results_list = []
for n in results:
    for m in results[n]:
        lp_results_list.append({'n': n, 'm': m,'lp-fractional':results[n][m]['fractional_values'], 'fractional time':results[n][m]['lp_fractional_time'],'fractional_cover': results[n][m]['lp_rounding_cover'],'lp_rounding': results[n][m]['lp_optimal'],'lp_rounding_time': results[n][m]['lp_rounding_time']  })
lp_results_df = pd.DataFrame(lp_results_list)


comparison_df = pd.merge(approx_results, lp_results_df, on=['n', 'm'])

# Approximation Factor = Size of Approximate Cover / LP Optimal
comparison_df['2 Factor Approx Factor (vs LP Optimal)'] = comparison_df['2 Factor Approximation'] / comparison_df['lp_rounding']

display(comparison_df)

,n,m,2 Factor Approximation,Approx Time,lp-fractional,fractional time,fractional_cover,lp_rounding,lp_rounding_time,2 Factor Approx Factor (vs LP Optimal)
0,10,10,8,0.000038,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, -0.0,...",0.027643,"[0, 4, 6, 7]",4.0,0.000010,2.00
1,10,20,10,0.000017,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...",0.005373,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",5.0,0.000008,2.00
2,10,30,10,0.000016,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...",0.003923,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",5.0,0.000012,2.00
3,10,40,10,0.000015,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...",0.004682,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",5.0,0.000006,2.00
4,20,20,14,0.000059,"[0.0, 1.0, 0.0, 0.0, 0.0, -0.0, 1.0, 1.0, 0.0,...",0.004306,"[1, 6, 7, 9, 12, 13, 14, 18]",8.0,0.000012,1.75
5,20,40,16,0.000043,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...",0.007615,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",10.0,0.000020,1.60
6,20,60,18,0.000040,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...",0.009706,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",10.0,0.000015,1.80
7,20,80,18,0.000057,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...",0.006786,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",10.0,0.000012,1.80
8,20,100,20,0.000049,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...",0.006687,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",10.0,0.000013,2.00
9,20,120,20,0.000045,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...",0.006245,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",10.0,0.000013,2.00


In [ ]:
comparison_df.to_csv('comparison_results.csv', index=False)